# generating HiC heatmap on the ALLHIC assembled chromosomes

* this is for blueskies project test graphs
* instructions: https://github.com/phasegenomics/juicebox_scripts

## 1. generating mapping bam file

In [140]:
REF=/workspace/hraczw/github/GA/Gillenia_genome/008.Allhic.scaffolding/groups.asm.fasta

In [141]:
HICR1_RAW=/workspace/hraczw/github/GA/Gillenia_genome/002.Fastp.trimming/R1.cleaned.specifiedAdapter.short.Q15.fq.gz
HICR2_RAW=/workspace/hraczw/github/GA/Gillenia_genome/002.Fastp.trimming/R2.cleaned.specifiedAdapter.short.Q15.fq.gz

In [142]:
module load ncbi-blast
module list

Currently Loaded Modulefiles:
 1) powerPlant/core    5) abatch/0.1        9) asub/2.2           
 2) texlive/20151117   6) Slurm/21.08.8-2  10) ncbi-blast/2.11.0  
 3) pandoc/1.19.2      7) perlbrew/0.76    11) bwa/0.7.17         
 4) git/2.21.0         8) perl/5.36.0      12) samtools/1.12      

Key:
auto-loaded  sticky  


In [143]:
mkdir 021.JuiceBox_visualisation

mkdir: cannot create directory ‘021.JuiceBox_visualisation’: File exists


: 1

In [144]:
WDR=021.JuiceBox_visualisation

In [1]:
ll

total 252373240
drwxrwsr-x.  4 hraczw powerplant         2118 Aug 11 13:16 002.Fastp.trimming
drwxrwsr-x.  3 hraczw powerplant         6972 Jul 30  2020 0031.Contamination.analysis
drwxrwsr-x.  3 hraczw powerplant         2453 Jan 23  2020 003.PG.QC
drwxrwsr-x. 10 hraczw powerplant         1109 Jul  5  2020 004.Scaffolding_10Xcontigs
drwxrwsr-x.  5 hraczw powerplant        18940 Sep  6  2020 005.GapFilling
drwxrwsr-x.  3 hraczw powerplant        15393 Sep  7  2020 006.SyntenyAnalysis
drwxrwsr-x.  3 hraczw powerplant          508 Sep 29  2020 007.TelomereCheck
drwxrwsr-x.  6 hraczw powerplant         3386 Aug  9 15:27 008.Allhic.scaffolding
drwxrwsr-x.  2 hraczw powerplant          747 Aug 17  2020 009.SyntenyBetweenAssemblies
-rw-rw-r--.  1 hraczw powerplant        55860 Sep  2  2020 01-09-2020_salsa_scaffolding.ipynb
-rw-rw-r--.  1 hraczw powerplant        30103 Sep  2  2020 01-09-2020_scaffhic_scaffolding.ipynb
drwxrwsr-x.  8 hraczw powerplant         1645 Aug 21  2020 010.PurgeHaplo

In [145]:
# mapping following in steps on ALLHIC wiki

module load bwa
module list

Currently Loaded Modulefiles:
 1) powerPlant/core    5) abatch/0.1        9) asub/2.2           
 2) texlive/20151117   6) Slurm/21.08.8-2  10) ncbi-blast/2.11.0  
 3) pandoc/1.19.2      7) perlbrew/0.76    11) bwa/0.7.17         
 4) git/2.21.0         8) perl/5.36.0      12) samtools/1.12      

Key:
auto-loaded  sticky  


In [146]:
sbatch << EOF
#!/bin/bash
#SBATCH -J bwa
#SBATCH -o ${WDR}/%J.out
#SBATCH -e ${WDR}/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=50G
#SBATCH --time=01:00:00

# load the module inside the job
module load bwa/0.7.17

bwa index $REF

EOF
#squeue

Submitted batch job 111126


In [147]:
module load samtools
module list

Currently Loaded Modulefiles:
 1) powerPlant/core    5) abatch/0.1        9) asub/2.2           
 2) texlive/20151117   6) Slurm/21.08.8-2  10) ncbi-blast/2.11.0  
 3) pandoc/1.19.2      7) perlbrew/0.76    11) bwa/0.7.17         
 4) git/2.21.0         8) perl/5.36.0      12) samtools/1.12      

Key:
auto-loaded  sticky  


In [148]:
sbatch << EOF
#!/bin/bash
#SBATCH -J map
#SBATCH -o ${WDR}/%J.out
#SBATCH -e ${WDR}/%J.err
#SBATCH --cpus-per-task=20
#SBATCH --mem=50G
#SBATCH --time=20:00:00

# load the module inside the job
module load samtools/1.12
module load bwa/0.7.17

bwa mem -5SP -t 20 $REF $HICR1_RAW $HICR2_RAW | samtools view -S -h -b -F 2316 > $WDR/mapped_allhic_assembly.bam

EOF

Submitted batch job 111127


## 2. generating .hic file for juicerbox

* using phasegenomics scripts: https://github.com/phasegenomics/juicebox_scripts

In [149]:
python /workspace/hraczw/github/programs/juicebox_scripts/juicebox_scripts/makeAgpFromFasta.py $REF $WDR/bilberry_allhic_assembly.agp

In [150]:
python /workspace/hraczw/github/programs/juicebox_scripts/juicebox_scripts/agp2assembly.py $WDR/bilberry_allhic_assembly.agp $WDR/bilberry_allhic_assembly.assembly

In [151]:
export PATH=/workspace/hraczw/github/programs/matlock/bin/:$PATH

In [152]:
sbatch << EOF
#!/bin/bash
#SBATCH -J map
#SBATCH -o ${WDR}/%J.out
#SBATCH -e ${WDR}/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=50G
#SBATCH --time=10:00:00

# load the module inside the job
export PATH=/workspace/hraczw/github/programs/matlock/bin/:$PATH

matlock bam2 juicer $WDR/mapped_allhic_assembly.bam $WDR/out_allhic_assembly.links.txt

EOF

Submitted batch job 111128


In [153]:
sbatch << EOF
#!/bin/bash
#SBATCH -J sort
#SBATCH -o ${WDR}/%J.out
#SBATCH -e ${WDR}/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=50G
#SBATCH --time=10:00:00

# load the module inside the job
export PATH=/workspace/hraczw/github/programs/matlock/bin/:$PATH

sort -k2,2 -k6,6 $WDR/out_allhic_assembly.links.txt > $WDR/out_allhic_assembly.sorted.links.txt

EOF

Submitted batch job 111129


In [139]:
sbatch << EOF
#!/bin/bash
#SBATCH -J convert2hic
#SBATCH -o ${WDR}/%J.out
#SBATCH -e ${WDR}/%J.err
#SBATCH --cpus-per-task=1
#SBATCH --mem=100G
#SBATCH --time=20:00:00

# load the module inside the job
export PATH=/workspace/hraczw/github/programs/matlock/bin/:$PATH

bash /workspace/hraczw/github/programs/3d-dna/visualize/run-assembly-visualizer.sh -p false $WDR/bilberry_allhic_assembly.assembly $WDR/out_allhic_assembly.sorted.links.txt

EOF

Submitted batch job 111124
